# Economic Data Dashboard Notebook


# http://127.0.0.1:8067/

# 1. Configuration and Initialization. (FredApi and Start Date)

In [19]:
# Configuration and Initialization
import pandas as pd
import numpy as np
from fredapi import Fred
import plotly.graph_objects as go

from plotly.subplots import make_subplots
import dash
import time
from dash import dcc, html
from dash.dependencies import Input, Output
import requests
import math
from datetime import datetime
from dateutil.relativedelta import relativedelta

# FRED API Configuration
api_key = '7227512392e5e5d2a2679a261d2bb3a9'
fred = Fred(api_key=api_key)

# Define the start and end dates
start = '2015-01-01'
end = '2024-07-01'


In [2]:
import investpy
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Get economic calendar data for the United States from the year 2020 onwards
economic_calendar_data = investpy.economic_calendar(
    countries=['united states'],
    from_date='01/01/2020',
    to_date='31/12/2024'
)

# Convert the data to a DataFrame
calendar_df = pd.DataFrame(economic_calendar_data)

# 2. Fetching Data from FRED

In [20]:
# Function to fetch data from FRED
def fetch_fred_data(series_id, api_key):
    url = f"https://api.stlouisfed.org/fred/series/observations?series_id={series_id}&api_key={api_key}&file_type=json"
    response = requests.get(url)
    data = response.json()
    
    if 'observations' not in data:
        print(f"Error fetching data for series {series_id}: {data}")
        return pd.DataFrame()  # Return an empty DataFrame in case of error
    
    return pd.DataFrame(data['observations'])


In [21]:
# Load data from CSV
def load_employment_data():
    file_path = os.path.join(os.getcwd(), 'data', 'employment_graph_1.csv')
    data = pd.read_csv(file_path, index_col=0, parse_dates=True)
    return data

# Function to generate the employment detailed graph
def generate_employment_detailed_graph_1():
    # Load data from CSV
    data = load_employment_data()

    # Filter data from January 2015 onwards for slider purposes
    data = data[data.index >= '2015-01-01']
    
    # Ensure data does not exceed current date
    data = data[data.index <= '2024-10-01']
    
    # Drop rows with NaN values
    data.dropna(inplace=True)
    
    # Determine the range for 30-month windows with an additional month for MoM calculation
    window_size = 30  # months to display
    extra_month = 1   # additional month for MoM
    total_window_size = window_size + extra_month  # total months in each window
    
    windows = []
    current_start = data.index.min()
    while True:
        current_end = current_start + relativedelta(months=total_window_size-1)
        window = data[(data.index >= current_start) & (data.index <= current_end)]
        if len(window) == total_window_size:
            windows.append(window)
        else:
            break  # Exit if the window does not have enough data
        current_start += relativedelta(months=1)
        if current_end >= data.index.max():
            break  # Prevent going beyond the latest date

    # If no windows found, return an empty figure
    if not windows:
        return go.Figure()
    
    # Function to prepare table data with correct MoM calculations and chronological order
    def prepare_table_data(window):
        # window is a DataFrame with 31 months (for 30 display months + 1 prior month)
        # Calculate MoM changes based on the prior month
        nfp_changes = window['Thousand Changes'].round(2)
        moM_changes = window['MoM % Change'].round(2)
        
        # Drop the first row (prior month) to get 30 months' data
        table_window = window.iloc[1:].copy()
        table_window['Thousand Changes'] = nfp_changes.iloc[1:].fillna(0).values
        table_window['MoM % Change'] = moM_changes.iloc[1:].fillna(0).values
        
        # Sort the DataFrame by date in ascending order (oldest on left)
        table_window = table_window.sort_index(ascending=True)
        
        # Round "NFP (Millions)" to 1 decimal place
        table_window['NFP'] = table_window['NFP'].round(1)
        
        # Prepare the DataFrame for the table, including NFP
        table_data = pd.DataFrame({
            "NFP (Millions)": table_window['NFP'].values,
            "Thousand Changes": table_window['Thousand Changes'].values,
            "Unemployment Rate (%)": table_window['Unemployment Rate (%)'].values,
            "MoM % Change": table_window['MoM % Change'].values
        }).T.round(2)
        
        # Extract the dates as column headers in ascending order
        dates_for_table = table_window.index.strftime("%y-%m")

        return table_data, dates_for_table

    # Prepare all table data for frames (will be used to create frames in slider)
    prepared_windows = [prepare_table_data(window) for window in windows]
    
    # Reverse the windows and prepared_windows lists to have newest first
    windows = windows[::-1]
    prepared_windows = prepared_windows[::-1]
    
    # Function to generate colors for the heatmap
    def get_row_heatmap_colors(row, row_name):
        vmin, vmax = row.min(), row.max()  # Get min and max values for normalization
        if vmin == vmax:
            norm = [0.5] * len(row)
        else:
            norm = [(value - vmin) / (vmax - vmin) if vmax != vmin else 0.5 for value in row]
        colors = []
        for value, n in zip(row, norm):
            if row_name == "NFP (Millions)":
                # Green shades for positive trends
                green_intensity = max(0, min(int(255 * n), 255))  # Clamp between 0 and 255
                colors.append(f'rgba(0, {green_intensity}, 0, 0.6)')
            elif row_name == "Thousand Changes":
                # Green shades for positive changes, red for negative
                if value >= 0:
                    green_intensity = max(0, min(int(255 * n), 255))
                    colors.append(f'rgba(0, {green_intensity}, 0, 0.6)')
                else:
                    red_intensity = max(0, min(int(255 * abs(n)), 255))
                    colors.append(f'rgba({red_intensity}, 0, 0, 0.6)')
            elif row_name == "Unemployment Rate (%)":
                # Red shades for higher unemployment rates
                red_intensity = max(0, min(int(255 * n), 255))
                colors.append(f'rgba({red_intensity}, 0, 0, 0.6)')
            elif row_name == "MoM % Change":
                # Green for negative changes (better), red for positive changes (worse)
                if value < 0:
                    green_intensity = max(0, min(int(255 * (-value / vmax)), 255))
                    colors.append(f'rgba(0, {green_intensity}, 0, 0.6)')
                elif value > 0:
                    red_intensity = max(0, min(int(255 * (value / vmax)), 255))
                    colors.append(f'rgba({red_intensity}, 0, 0, 0.6)')
                else:
                    colors.append('rgba(128, 128, 128, 0.6)')  # Gray for no change
            else:
                # Default to gray if row_name is unrecognized
                colors.append('rgba(128, 128, 128, 0.6)')
        return colors
    
    # Generate colors for the initial table
    initial_table_data, initial_dates_for_table = prepared_windows[0]
    initial_colors = [get_row_heatmap_colors(row, row_name) for row_name, row in initial_table_data.iterrows()]
    initial_flat_colors = list(map(list, zip(*initial_colors)))  # Transpose for Plotly
    
    # Determine the initial shading window
    initial_window = windows[0]
    window_start = initial_window.index.min()
    window_end = initial_window.index.max()
    
    # Create subplots: 2 rows x 2 columns
    fig = make_subplots(
        rows=2, cols=2, 
        row_heights=[0.60, 0.40],  # 60% for charts, 40% for table
        specs=[
            [{'type': 'bar'}, {'type': 'scatter'}],  # First row: bar and scatter plots
            [{'type': 'table', 'colspan': 2}, None]  # Second row: table spanning both columns
        ],
        subplot_titles=(
            'Change in Nonfarm Payroll (Thousands)', 
            'Unemployment Rate (%)'
        ),
        vertical_spacing=0.05  # Smaller gap between plots and table
    )
    
    # Add the Bar chart (Trace 0)
    bar_trace = go.Bar(
        x=data.index,  # All data since 2015
        y=data['Thousand Changes'], 
        name='Thousand Changes',
        marker_color='rgba(70, 130, 180, 0.7)',  # SteelBlue with transparency
        hovertemplate='Date: %{x}<br>Change: %{y} Thousands',
        text=data['Thousand Changes'].round(2),
        textposition='outside',
        textfont=dict(size=8)
    )
    fig.add_trace(bar_trace, row=1, col=1)
    
    # Add the 3-Month Moving Average line (Trace 1)
    ma_trace = go.Scatter(
        x=data.index, 
        y=data['3mo MA'], 
        mode='lines', 
        name='3-Month MA', 
        line=dict(color='rgba(255, 165, 0, 0.9)', width=2),  # Orange with slight transparency
        hovertemplate='Date: %{x}<br>3-Month MA: %{y:.2f} Thousands'
    )
    fig.add_trace(ma_trace, row=1, col=1)
    
    # Add the Unemployment Rate line (Trace 2)
    unrate_trace = go.Scatter(
        x=data.index, 
        y=data['Unemployment Rate (%)'], 
        mode='lines', 
        name='Unemployment Rate (%)', 
        line=dict(color='rgba(34, 139, 34, 0.9)', width=2),  # ForestGreen with slight transparency
        hovertemplate='Date: %{x}<br>Unemployment Rate: %{y}%'
    )
    fig.add_trace(unrate_trace, row=1, col=2)
    
    # Add initial table (Trace 3)
    table = go.Table(
        header=dict(
            values=["Items"] + list(initial_dates_for_table),
            fill_color='black',
            font=dict(color='white', size=8),  # Header font size set to 8
            align='center',
            height=30
        ),
        cells=dict(
            values=[
                initial_table_data.index,  # Row labels
                *[initial_table_data[col] for col in initial_table_data.columns]  # Column data
            ],
            fill_color=[['black'] * len(initial_dates_for_table)] + initial_flat_colors,
            font=dict(color='white', size=8),  # Cell font size set to 8
            align=['left', 'center'],
            height=25
        )
    )
    fig.add_trace(table, row=2, col=1)
    
    # Add initial shading to both subplots
    initial_shaded_shape_plot1 = dict(
        type="rect",
        xref="x1",
        yref="y1",
        x0=window_start,
        x1=window_end,
        y0=data['Thousand Changes'].min(),
        y1=data['Thousand Changes'].max(),
        fillcolor="grey",
        opacity=0.2,
        line_width=0
    )
    
    initial_shaded_shape_plot2 = dict(
        type="rect",
        xref="x2",  # x2 refers to the second subplot's x-axis
        yref="y2",
        x0=window_start,
        x1=window_end,
        y0=data['Unemployment Rate (%)'].min(),
        y1=data['Unemployment Rate (%)'].max(),
        fillcolor="grey",
        opacity=0.2,
        line_width=0
    )
    
    fig.update_layout(
        shapes=[initial_shaded_shape_plot1, initial_shaded_shape_plot2]
    )
    
    # Generate frames for each 30-month window
    frames = []
    for i, (window, (table_data, window_dates)) in enumerate(zip(windows, prepared_windows)):
        # Generate colors for the table (retain original colormap)
        window_colors = [get_row_heatmap_colors(row, row_name) for row_name, row in table_data.iterrows()]
        window_flat_colors = list(map(list, zip(*window_colors)))  # Transpose
        
        # Determine shading boundaries
        shading_start = window.index.min()
        shading_end = window.index.max()
        # Shading covers entire y-axis ranges, so use overall min and max
        shading_y1_plot1 = data['Thousand Changes'].min()
        shading_y2_plot1 = data['Thousand Changes'].max()
        shading_y1_plot2 = data['Unemployment Rate (%)'].min()
        shading_y2_plot2 = data['Unemployment Rate (%)'].max()
        
        # Create shaded shapes for the current window
        shaded_shape_plot1 = dict(
            type="rect",
            xref="x1",
            yref="y1",
            x0=shading_start,
            x1=shading_end,
            y0=shading_y1_plot1,
            y1=shading_y2_plot1,
            fillcolor="grey",
            opacity=0.2,
            line_width=0
        )
        
        shaded_shape_plot2 = dict(
            type="rect",
            xref="x2",  # x2 for the second subplot
            yref="y2",
            x0=shading_start,
            x1=shading_end,
            y0=shading_y1_plot2,
            y1=shading_y2_plot2,
            fillcolor="grey",
            opacity=0.2,
            line_width=0
        )
        
        # Create a frame with the updated table and shaded shapes
        frame = go.Frame(
            data=[
                # Update the Table (Trace index 3)
                go.Table(
                    header=dict(
                        values=["Items"] + list(window_dates),
                        fill_color='black',
                        font=dict(color='white', size=8),  # Header font size set to 8
                        align='center',
                        height=30
                    ),
                    cells=dict(
                        values=[
                            table_data.index,  # Row labels
                            *[table_data[col] for col in table_data.columns]  # Column data
                        ],
                        fill_color=[['black'] * len(window_dates)] + window_flat_colors,
                        font=dict(color='white', size=8),  # Cell font size set to 8
                        align=['left', 'center'],
                        height=25
                    )
                )
            ],
            layout=go.Layout(
                shapes=[shaded_shape_plot1, shaded_shape_plot2]  # Update the shading
            ),
            name=str(i),      # Each frame needs a unique name
            traces=[3]        # Specify that only trace index 3 (the table) is updated
        )
        frames.append(frame)
    
    fig.frames = frames
    
    # Define slider steps
    slider_steps = []
    for i, window in enumerate(windows):
        step = dict(
            method="animate",
            args=[
                [str(i)],
                dict(
                    mode="immediate", 
                    frame=dict(duration=0, redraw=True), 
                    transition=dict(duration=0)
                )
            ],
            label=window.index.min().strftime("%y-%m")
        )
        slider_steps.append(step)
    
    # Reverse the slider steps to have oldest dates on the left and newest on the right
    slider_steps = slider_steps[::-1]
    
    # Add slider to the layout, starting on the right (newest window)
    sliders = [dict(
        active=len(slider_steps)-1,  # Start slider at the newest window (rightmost)
        currentvalue={"prefix": "Start Date: "},
        pad={"t": 50},
        steps=slider_steps
    )]
    
    # Update layout with font sizes, y-axis range, color scheme, and slider
    fig.update_layout(
        sliders=sliders,
        title="",
        yaxis=dict(
            title="Change in NFP (Thousands)",
            range=[-200, 3000],  # Set y-axis range from -200 to 1000
            titlefont=dict(size=10),
            tickfont=dict(size=8)
        ),
        yaxis2=dict(
            title="Unemployment Rate (%)",
            titlefont=dict(size=10),
            tickfont=dict(size=8)
        ),
        xaxis=dict(
            range=[data.index.min(), data.index.max()],
            tickfont=dict(size=8),
            titlefont=dict(size=10)
        ),
        xaxis2=dict(
            range=[data.index.min(), data.index.max()],
            tickfont=dict(size=8),
            titlefont=dict(size=10)
        ),
        template="plotly_dark",  # Retain the dark theme
        font=dict(
            color="white",  # Keep font color white for dark background
            size=8  # Set default font size to 8
        ),
        title_x=0.5,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="center",
            x=0.5,
            font=dict(size=8, color="white")  # Ensure legend text is white
        ),
        height=900,    # Adjusted height
        width=1000,    # Increased width to accommodate 30 columns
        margin=dict(t=80, b=50, l=20, r=20)  # Increased bottom margin for slider and table
    )
    
    # Add source annotation at the bottom with hyperlinks
    fig.add_annotation(
        text=('Source: Federal Reserve Economic Data (FRED). '
              '<a href="https://fred.stlouisfed.org/series/PAYEMS" style="color: white">NFP</a>, '
              '<a href="https://fred.stlouisfed.org/series/UNRATE" style="color: white">Unemployment Rate</a>'),
        font=dict(size=12, color="white"),
        xref="paper", yref="paper",
        x=0.5, y=-0.015,
        showarrow=False,
        xanchor="center"
    )
    
    # Add a legend or note about MoM % Change interpretation
    fig.add_annotation(
        text="Note: For MoM Employment Changes, lower values are better.",
        font=dict(size=10, color="white"),
        xref="paper", yref="paper",
        x=0.5, y=-0.25,
        showarrow=False,
        xanchor="center"
    )
    
    return fig

/var/folders/lg/dbmt0tnx1h74vny5m2398l980000gr/T/ipykernel_72992/2689285243.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/lg/dbmt0tnx1h74vny5m2398l980000gr/T/ipykernel_72992/2689285243.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/lg/dbmt0tnx1h74vny5m2398l980000gr/T/ipykernel_72992/2689285243.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
Cell In[18], line 175, in update_employment_detailed_graph_1(selected_data='UNEMPLOYMENT')
    170 @app.callback(
    171     Output('employment-detailed-graph-1', 'figure'),
    172     [Input('data-dropdown', 'value')]
    173 )
    174 def update_employment_detailed_graph_1(selected_data):
--> 175     return generate_employment_detailed_graph_1() if selected_data == 'UNEMPLOYMENT' else go.Figure()
        selected_data == 'UNEMPLOYMENT' = True
        selected_data = 'UNEMPLOYMENT'
        go = <module 'plotly.graph_objects' from '/opt/anaconda3/lib/python3.11/site-packages/plotly/graph_objects/__init__.py'>

Cell In[21], line 10, in generate_employment_detailed_graph_1()
      8 def generate_employment_detailed_graph_1():
      9     # Load data from CSV
---> 10     data = load_employment_data()
     12     # Filter data from Januar

/var/folders/lg/dbmt0tnx1h74vny5m2398l980000gr/T/ipykernel_72992/2166681660.py:31: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/lg/dbmt0tnx1h74vny5m2398l980000gr/T/ipykernel_72992/2166681660.py:32: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/lg/dbmt0tnx1h74vny5m2398l980000gr/T/ipykernel_72992/2166681660.py:33: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/lg/dbmt0tnx1h74vny5m2398l980000gr/T/

No data found for Mining and logging


In [5]:
from datetime import datetime
def generate_employment_detailed_graph_2():
    # Initialize Fred API
    fred = Fred(api_key='7227512392e5e5d2a2679a261d2bb3a9')
    
    # Fetch the data
    start_date = '2000-01-01'
    end_date = datetime.today().strftime('%Y-%m-%d')
    
    fed_funds_rate = fred.get_series('EFFR', start_date, end_date)
    unemployment_rate = fred.get_series('UNRATE', start_date, end_date)
    job_openings_rate = fred.get_series('JTSJOR', start_date, end_date)
    
    # Create a subplot figure with 2 rows and 1 column
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                        row_heights=[0.4, 0.6], 
                        vertical_spacing=0.1)
    
    # Plot Federal Funds Rate on the top subplot
    trace1 = go.Scatter(
        x=fed_funds_rate.index, y=fed_funds_rate.values,
        mode='lines',
        name='Federal Funds Rate',
        line=dict(color='cyan', width=2)
    )
    
    fig.add_trace(trace1, row=1, col=1)
    
    # Plot U-3 Unemployment Rate and Job Openings Rate on the bottom subplot
    trace2 = go.Scatter(
        x=unemployment_rate.index, y=unemployment_rate.values,
        mode='lines',
        name='U-3 US Unemployment Rate',
        line=dict(color='orange', width=2)
    )
    
    trace3 = go.Scatter(
        x=job_openings_rate.index, y=job_openings_rate.values,
        mode='lines',
        name='US Job Openings Rate',
        line=dict(color='white', width=2)
    )
    
    fig.add_trace(trace2, row=2, col=1)
    fig.add_trace(trace3, row=2, col=1)
    
    # Add recession bands (you can manually input recession periods)
    recession_bands = [
        {'start': '2001-03-01', 'end': '2001-11-01'},
        {'start': '2007-12-01', 'end': '2009-06-01'},
        {'start': '2020-02-01', 'end': '2020-04-01'}
    ]
    
    # Add recession bands to both subplots, matching the correct y-ranges
    for band in recession_bands:
        fig.add_shape(
            type="rect",
            xref="x", x0=band['start'], x1=band['end'],
            yref="y1", y0=0, y1=7,  # Matches the y-axis range of the Federal Funds Rate
            fillcolor="red", opacity=0.3, line_width=0
        )
        fig.add_shape(
            type="rect",
            xref="x", x0=band['start'], x1=band['end'],
            yref="y2", y0=0, y1=15,  # Matches the y-axis range of the Unemployment Rate and Job Openings Rate
            fillcolor="red", opacity=0.3, line_width=0
        )
    
    # Add annotations for the most recent data points
    annotations = [
        dict(
            x=fed_funds_rate.index[-1], y=fed_funds_rate.values[-1],
            xref='x', yref='y1',
            text=f'{fed_funds_rate.values[-1]:.2f}%',
            showarrow=True, arrowhead=1, ax=20, ay=-30,
            font=dict(color='cyan')
        ),
        dict(
            x=unemployment_rate.index[-1], y=unemployment_rate.values[-1],
            xref='x', yref='y2',
            text=f'{unemployment_rate.values[-1]:.2f}%',
            showarrow=True, arrowhead=1, ax=20, ay=-30,
            font=dict(color='orange')
        ),
        dict(
            x=job_openings_rate.index[-1], y=job_openings_rate.values[-1],
            xref='x', yref='y2',
            text=f'{job_openings_rate.values[-1]:.2f}%',
            showarrow=True, arrowhead=1, ax=20, ay=-30,
            font=dict(color='white')
        )
    ]
    
    # Add layout settings
    fig.update_layout(
        title='',
        height=500,  # Increase the overall graph height
        template="plotly_dark",
        font=dict(color='white'),
        annotations=annotations,
        yaxis1=dict(title='Federal Funds Rate (%)', range=[0, 7], tickcolor='white', ticks="outside"),
        yaxis2=dict(title='Rate (%)', range=[0, 15], tickcolor='white', ticks="outside"),
        xaxis2=dict(title='Date', tickcolor='white', ticks="outside"),
        
        # Legend placed horizontally under the title
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.05,
            xanchor="center",
            x=0.5
        ),
    )
    
    return fig


In [6]:
def generate_employment_detailed_graph_3():
    # Initialize FRED API (replace with your API key)
    fred = Fred(api_key=api_key)
    
    # Fetch data since 2022
    start_date = '2022-01-01'
    nonfarm_payrolls = fred.get_series('PAYEMS', start_date=start_date)
    unemployment_rate = fred.get_series('UNRATE', start_date=start_date)
    
    # Calculate the monthly change and 3-month moving average for nonfarm payrolls
    nonfarm_payrolls_monthly_change = nonfarm_payrolls.diff()
    nonfarm_3m_avg = nonfarm_payrolls_monthly_change.rolling(window=3).mean()
    
    # Filter out data only since 2022
    nonfarm_3m_avg = nonfarm_3m_avg[start_date:]
    unemployment_rate = unemployment_rate[start_date:]
    
    # Prepare the plot
    fig = make_subplots(rows=1, cols=2, subplot_titles=('Total Nonfarm Payrolls (Monthly Change): 3M Average', 'U3 Unemployment Rate'))
    
    # Plot nonfarm payrolls (3M average)
    fig.add_trace(go.Bar(x=nonfarm_3m_avg.index, y=nonfarm_3m_avg, name='Total Nonfarm', marker_color='white'), row=1, col=1)
    
    # Plot U3 unemployment rate
    fig.add_trace(go.Bar(x=unemployment_rate.index, y=unemployment_rate, name='U3 Unemployment Rate', marker_color='white'), row=1, col=2)
    
    # Update layout with dark theme
    fig.update_layout(
        title='Labor Market',
        template='plotly_dark',
        xaxis_title='Date',
        yaxis_title='Thousands',
        yaxis2_title='Percentage',
        yaxis2=dict(title='Percentage', range=[3.0, 4.5]),  # Set y-axis range for the U3 Unemployment Rate
        showlegend=False,
        height=500,
        width=1000
    )

    return fig


In [7]:
def generate_employment_detailed_graph_4(observation_start='2020-01-01'):
    # Initialize FRED API
    fred = Fred(api_key=api_key)

    # Employment series for Nonfarm and Sub-categories (sample FRED series IDs assumed)
    series_ids = {
        'Total Nonfarm': 'PAYEMS',  # Adjust if different
        'Total Private': 'USPRIV',
        'Goods-producing': 'USGOOD',
        '  Mining and Logging': 'USMINE',
        '  Construction': 'USCONS',
        '  Manufacturing': 'MANEMP',
        'Private Service-providing': 'CES0800000001',
        '  Trade, Transportation, and Utilities': 'USTPU',
        '  Information': 'USINFO',
        '  Financial Activities': 'USFIRE',
        '  Professional and Business Services': 'USPBS',
        '  Education and Health Services': 'USEHS',
        '  Leisure and Hospitality': 'USLAH',
        '  Other Services': 'USSERV',
        'Government': 'USGOVT'
    }

    bold_categories = ['Total Nonfarm', 'Total Private', 'Goods-producing', 'Private Service-providing']

    data = {}
    for name, series_id in series_ids.items():
        if series_id:
            while True:
                try:
                    # Fetch data from FRED with the specified observation start date
                    series_data = fred.get_series(series_id, observation_start=observation_start).resample('M').mean()
                    break  # Exit loop if successful
                except ValueError as e:
                    if "Too Many Requests" in str(e):
                        print("Rate limit exceeded. Waiting before retrying...")
                        time.sleep(60)  # Wait for a minute before retrying
                    else:
                        raise e

            # Calculate MoM difference
            series_data = series_data.diff()
            data[name] = series_data

    df = pd.DataFrame(data)
    df.dropna(how='all', inplace=True)
    df.fillna(0, inplace=True)
    df = df.transpose()

    # Function to generate colors for heatmap per item
    def get_row_heatmap_colors(row):
        vmin, vmax = row.min(), row.max()
        return [get_heatmap_color(value, vmin, vmax) for value in row]

    # Function to get color based on normalized value
    def get_heatmap_color(value, vmin, vmax):
        norm_value = (value - vmin) / (vmax - vmin) if vmax - vmin != 0 else 0.5
        if value > 0:
            return f'rgba(0, {255 * norm_value}, 0, 0.6)'  # Green for positive values
        else:
            return f'rgba({255 * (1 - norm_value)}, 0, 0, 0.6)'  # Red for negative values

    # Apply color mapping to each row of the DataFrame
    colors = [get_row_heatmap_colors(row) for _, row in df.iterrows()]

    # Transpose color array to match Plotly's color format
    flat_colors = list(map(list, zip(*colors)))

    # Prepare table column headers and formatting for bold categories
    formatted_categories = []
    for category in df.index:
        if category in bold_categories:
            formatted_categories.append(f"<b>{category}</b>")
        elif category.startswith('  '):
            formatted_categories.append(f"&nbsp;&nbsp;&nbsp;{category.strip()}")
        else:
            formatted_categories.append(category)

    # Define the size of each window (30 months) and step (4 months)
    window_size = 30  # months to display
    step_size = 4     # months to step each window
    total_months = len(df.columns)
    total_windows = math.ceil((total_months - window_size) / step_size) + 1

    # Create windows of 30 months starting every 4 months
    windows = []
    for window_num in range(total_windows):
        start_idx = window_num * step_size
        end_idx = start_idx + window_size
        if end_idx > total_months:
            end_idx = total_months
            start_idx = end_idx - window_size
            if start_idx < 0:
                start_idx = 0
        window_dates = df.columns[start_idx:end_idx]
        window_data = df.iloc[:, start_idx:end_idx]
        windows.append((window_dates, window_data))

    # Function to prepare table data and colors for a given window
    def prepare_frame(window_dates, window_data):
        # Format the date labels
        formatted_window_dates = [date.strftime("%y-%m") for date in window_dates]
        
        # Prepare table data
        table_values = [formatted_categories]  # First row: Categories
        for col in window_dates:
            table_values.append(window_data[col].round(1))
        
        # Apply color mapping for the current window
        window_colors = []
        for i, row in enumerate(window_data.iterrows()):
            row_colors = get_row_heatmap_colors(window_data.iloc[i])
            window_colors.append(row_colors)

        # Header colors
        header_colors = ['black'] * (len(formatted_window_dates) + 1)
        
        # Combine header and cell colors
        color_fill_table = [header_colors] + list(map(list, zip(*window_colors)))
        
        # Create a table trace
        table = go.Table(
            columnwidth=[200] + [80 for _ in range(len(formatted_window_dates))],  # Adjust column widths as needed
            header=dict(
                values=["<b>Category</b>"] + list(formatted_window_dates),
                fill_color='black',
                font=dict(color='white', size=8),
                align='center',
                height=20
            ),
            cells=dict(
                values=[
                    table_values[0],  # formatted_categories
                    *table_values[1:]  # Data for each category
                ],
                fill=dict(color=color_fill_table),
                font=dict(color='white', size=8),
                align=['left'] + ['center'] * len(formatted_window_dates),
                height=20
            )
        )
        return table

    # Initialize the figure with the last window's table (show newest data first)
    initial_dates, initial_data = windows[-1]  # Start with the last (newest) window
    initial_table = prepare_frame(initial_dates, initial_data)

    # Create subplots: 1 row for the table
    fig = make_subplots(
        rows=1, cols=1,
        specs=[[{"type": "table"}]],
        subplot_titles=['Employment Detailed Data'],
        vertical_spacing=0.05
    )

    # Add the initial table
    fig.add_trace(initial_table, row=1, col=1)

    # Create frames for each window
    frames = []
    for i, (window_dates, window_data) in enumerate(windows):
        frame_table = prepare_frame(window_dates, window_data)
        frame = go.Frame(
            data=[frame_table],
            name=str(i)
        )
        frames.append(frame)

    fig.frames = frames

    # Define slider steps with labels at every step
    slider_steps = []
    for i, (window_dates, _) in enumerate(windows):
        # Label as the first date in the window
        label = window_dates[0].strftime("%y-%m")
        step = dict(
            method='animate',
            args=[
                [str(i)],
                dict(
                    mode='immediate',
                    frame=dict(duration=500, redraw=True),
                    transition=dict(duration=0)
                )
            ],
            label=label
        )
        slider_steps.append(step)

    # Set the slider to start at the last step (right side) but move leftward for the user
    sliders = [dict(
        active=len(slider_steps) - 1,  # Start at the newest data
        currentvalue={"prefix": "Start Date: "},
        pad={"t": 50},
        steps=slider_steps
    )]

    # Update layout with sliders and annotations
    fig.update_layout(
        sliders=sliders,
        title='',
        template='plotly_dark',
        width=1000,
        height=850,
        margin=dict(l=20, r=20, t=80, b=50),
        annotations=[
            dict(
                text='Source: <a href="https://fred.stlouisfed.org/">FRED</a>',
                x=0.5,
                y=-0.15,
                xref='paper',
                yref='paper',
                showarrow=False,
                font=dict(size=8, color="white"),
                xanchor='center'
            )
        ]
    )

    return fig

In [8]:
def generate_employment_detailed_graph_5(observation_start='2020-01-01'):
    # Initialize FRED API
    fred = Fred(api_key=api_key)

    # Employment series
    series_ids = {
        'Total Nonfarm Private Payroll Employment': 'ADPMNUSNERSA',
        'Goods Producing': '',
        '  Construction': 'ADPMINDCONNERSA',
        '  Manufacturing': 'ADPMINDMANNERSA',
        '  Natural Resources and Mining': 'ADPWINDNRMINNERSA',
        'Services': '',
        '  Education and Health Services': 'ADPMINDEDHLTNERNSA',
        '  Financial Activities': 'ADPMINDFINNERNSA',
        '  Information': 'ADPWINDINFONERSA',
        '  Leisure and Hospitality': 'ADPMINDLSHPNERNSA',
        '  Other Services': 'ADPWINDOTHSRVNERSA',
        '  Professional and Business Services': 'ADPWINDPROBUSNERSA',
        '  Trade, Transportation and Utilities': 'ADPWINDTTUNERSA',
        'Non Farm Private Employment': '',
        '  1-19 Employees': 'ADPMES1T19ENERSA',
        '  20-49 Employees': 'ADPMES20T49ENERSA',
        '  50-249 Employees': 'ADPMES50T249ENERSA',
        '  250-499 Employees': 'ADPMES250T499ENERSA',
        '  500+ Employees': 'ADPMES500PENERSA'
    }

    bold_categories = [
        'Total Nonfarm Private Payroll Employment', 'Goods Producing', 'Services', 'Non Farm Private Employment'
    ]

    data = {}
    for name, series_id in series_ids.items():
        if series_id:  # Check if the series_id is available (ignore bold categories)
            while True:
                try:
                    # Fetch data from FRED with the specified observation start date
                    series_data = fred.get_series(series_id, observation_start=observation_start).resample('M').mean()
                    break  # Exit the loop if successful
                except ValueError as e:
                    if "Too Many Requests" in str(e):
                        print("Rate limit exceeded. Waiting before retrying...")
                        time.sleep(60)  # Wait for a minute before retrying
                    else:
                        raise e

            # Calculate the difference as new - old
            series_data = series_data.diff() / 1000  # MoM difference in thousands (new - old)
            data[name] = series_data

    # Create a DataFrame from the retrieved data
    df = pd.DataFrame(data)

    # Drop rows with all NaN values to clean up the DataFrame
    df.dropna(how='all', inplace=True)

    # Fill NA values in the table to avoid errors
    df.fillna(0, inplace=True)

    # Transpose the DataFrame to ensure categories are rows and dates are columns
    df_table = df.T

    # Define window size and step for the slider
    window_size = 30  # months to display
    step_size = 4     # months to step each window
    total_months = len(df_table.columns)
    total_windows = math.ceil((total_months - window_size) / step_size) + 1  # Ensure all data is captured

    # Create sliding windows of data
    windows = []
    for i in range(total_windows):
        start_idx = i * step_size
        end_idx = start_idx + window_size
        if end_idx > total_months:
            end_idx = total_months
            start_idx = end_idx - window_size
            if start_idx < 0:
                start_idx = 0
        window_dates = df_table.columns[start_idx:end_idx]
        window_data = df_table.iloc[:, start_idx:end_idx]
        windows.append((window_dates, window_data))

    # Generate heatmap colors for rows
    def get_row_heatmap_colors(row):
        vmin, vmax = row.min(), row.max()
        return [get_heatmap_color(value, vmin, vmax) for value in row]

    def get_heatmap_color(value, vmin, vmax):
        norm_value = (value - vmin) / (vmax - vmin) if vmax - vmin != 0 else 0.5
        if value > 0:
            return f'rgba(0, {255 * norm_value}, 0, 0.6)'  # Green for positive values
        else:
            return f'rgba({255 * (1 - norm_value)}, 0, 0, 0.6)'  # Red for negative values

    # Prepare data for the first window
    def prepare_frame(window_dates, window_data):
        # Format the column headers (dates)
        date_columns = [date.strftime("%y-%m") for date in window_dates]

        # Apply color mapping to rows
        colors = [get_row_heatmap_colors(row) for row in window_data.values]
        flat_colors = list(map(list, zip(*colors)))

        # Prepare colors for the table
        color_fill = [['black'] * len(date_columns)] * 1  # Header colors
        color_fill += flat_colors  # Cell colors

        # Format categories for table
        formatted_categories = []
        for category in window_data.index:
            if category in bold_categories:
                formatted_categories.append(f"<b>{category}</b>")
            elif category.startswith('  '):  # Indent subcategories
                formatted_categories.append(f"&nbsp;&nbsp;&nbsp;{category.strip()}")
            else:
                formatted_categories.append(category)

        # Create the table
        return go.Table(
            columnwidth=[200] + [80 for _ in range(len(date_columns))],
            header=dict(
                values=["Category"] + date_columns,
                fill_color='black',
                font=dict(color='white', size=8),  # Set font size to 8
                align='center'
            ),
            cells=dict(
                values=[formatted_categories] + [window_data[col].round(1) for col in window_data.columns],
                fill=dict(color=color_fill),
                font=dict(color='white', size=8),  # Set font size to 8 for cells
                align='left',
                height=30  # Adjust height if needed
            )
        )

    # Initialize the figure with the latest window (newest data)
    initial_dates, initial_data = windows[-1]  # Start with the last (newest) window
    initial_table = prepare_frame(initial_dates, initial_data)

    # Create figure
    fig = go.Figure(data=[initial_table])

    # Add animation frames for the slider
    frames = []
    for i, (window_dates, window_data) in enumerate(windows):
        frames.append(go.Frame(data=[prepare_frame(window_dates, window_data)], name=str(i)))

    fig.frames = frames

    # Create slider steps
    slider_steps = []
    for i, (window_dates, _) in enumerate(windows):
        label = window_dates[0].strftime("%y-%m")
        slider_steps.append(dict(
            method='animate',
            args=[[str(i)], dict(mode='immediate', frame=dict(duration=500, redraw=True), transition=dict(duration=0))],
            label=label
        ))

    # Create and configure the slider
    sliders = [dict(
        active=len(slider_steps) - 1,  # Start at the newest data (right)
        currentvalue={"prefix": "Start Date: "},
        pad={"t": 50},
        steps=slider_steps
    )]

    # Update layout with the slider and other configuration
    fig.update_layout(
        sliders=sliders,
        width=1000,
        height=850,
        template='plotly_dark',
        margin=dict(l=20, r=20, t=50, b=50),
        annotations=[
            dict(
                text="Source: Federal Reserve Economic Data (FRED) | Note: MoM Change (New - Old) in Thousands",
                x=0.5,
                y=-0.01,
                xref='paper',
                yref='paper',
                showarrow=False,
                font=dict(size=8, color="white"),  # Font size 8 for annotation
            )
        ]
    )

    return fig

In [9]:
def generate_employment_detailed_table_1(observation_start='2015-01-01'):
    # Initialize FRED API
    fred = Fred(api_key=api_key)

    # Employment series for Nonfarm, Sub-categories, and Unemployment-related categories
    series_ids = {
        'Job Losers': 'LNS13023621',
        'Job Leavers': 'LNS13023705',
        'Reentrants': 'LNS13023557',
        'New Entrants': 'LNS13023569'
    }

    # Retrieve data for each unemployment category
    data = {}
    for name, series_id in series_ids.items():
        if series_id:
            while True:
                try:
                    series_data = fred.get_series(series_id, observation_start=observation_start).resample('M').mean()
                    break
                except ValueError as e:
                    if "Too Many Requests" in str(e):
                        print("Rate limit exceeded. Waiting before retrying...")
                        time.sleep(60)
                    else:
                        raise e
            series_data = series_data.diff()
            data[name] = series_data

    # Create a DataFrame from the retrieved data
    df = pd.DataFrame(data)
    df.dropna(how='all', inplace=True)
    df.fillna(0, inplace=True)
    df = df.transpose()

    # Adjust slider step size to 6 months instead of 4
    window_size = 30  # months to display
    step_size = 6     # months to step each window
    total_months = len(df.columns)
    total_windows = math.ceil((total_months - window_size) / step_size) + 1

    windows = []
    for window_num in range(total_windows):
        start_idx = window_num * step_size
        end_idx = start_idx + window_size
        if end_idx > total_months:
            end_idx = total_months
            start_idx = end_idx - window_size  # Adjust start_idx to have full window
            if start_idx < 0:
                start_idx = 0
        window_dates = df.columns[start_idx:end_idx]
        window_data = df.iloc[:, start_idx:end_idx]
        windows.append((window_dates, window_data))

    # Format category names (bold if needed)
    formatted_categories = [f"<b>{category}</b>" if category in ['Total Nonfarm', 'Total Private', 'Goods-producing'] else category for category in df.index]

    # Custom function for row-based color mapping
    def get_row_heatmap_colors(row, category_name):
        vmin, vmax = row.min(), row.max()
        return [get_heatmap_color(value, vmin, vmax, category_name) for value in row]

    def get_heatmap_color(value, vmin, vmax, category):
        norm_value = (value - vmin) / (vmax - vmin)
        if category in ['Job Losers', 'Job Leavers']:
            # For Job Losers and Leavers, lower values are better (green), higher values are worse (red)
            if value > 0:
                return f'rgba({255 * norm_value}, 0, 0, 0.6)'  # Red for higher values (worse)
            else:
                return f'rgba(0, {255 * (1 + norm_value)}, 0, 0.6)'  # Green for lower values (better)

        elif category in ['Reentrants', 'New Entrants']:
            # For Reentrants and New Entrants, higher values are better (green), lower values are worse (red)
            if value > 0:
                return f'rgba(0, {255 * norm_value}, 0, 0.6)'  # Green for higher values (better)
            else:
                return f'rgba({255 * (1 - norm_value)}, 0, 0, 0.6)'  # Red for lower values (worse)

    # Prepare frame with the updated color mapping logic
    def prepare_frame(window_dates, window_data):
        formatted_window_dates = [date.strftime("%y-%m") for date in window_dates]
        table_values = [formatted_categories]  # First row: Categories
        for col in window_dates:
            table_values.append(window_data[col].round(1))

        # Generate colors for each row (category)
        colors = [get_row_heatmap_colors(window_data.loc[category], category) for category in window_data.index]

        # Flatten the colors for the table
        flat_colors = list(zip(*colors))

        # Prepare the colors for Plotly Table
        color_fill_table = [['black'] * len(window_dates)]  # Header colors
        color_fill_table += flat_colors  # Cell colors

        table = go.Table(
            header=dict(
                values=["Category"] + list(formatted_window_dates),
                fill_color='black',
                font=dict(color='white', size=8),
                align='center',
                height=20
            ),
            cells=dict(
                values=[table_values[0], *table_values[1:]],
                fill=dict(color=color_fill_table),
                font=dict(color='white', size=8),
                align=['left'] + ['center'] * len(formatted_window_dates),
                height=20
            )
        )
        return table

    # Create the initial plot
    initial_dates, initial_data = windows[-1]  # Start with the last (newest) window
    initial_table = prepare_frame(initial_dates, initial_data)

    fig = make_subplots(rows=1, cols=1, specs=[[{"type": "table"}]], vertical_spacing=0.05)
    fig.add_trace(initial_table, row=1, col=1)

    frames = []
    for i, (window_dates, window_data) in enumerate(windows):
        frame_table = prepare_frame(window_dates, window_data)
        frame = go.Frame(data=[frame_table], name=str(i))
        frames.append(frame)

    fig.frames = frames

    # Define the slider steps, moving from left (oldest) to right (newest)
    slider_steps = []
    for i, (window_dates, _) in enumerate(windows):
        label = window_dates[0].strftime("%y-%m")
        step = dict(
            method='animate',
            args=[
                [str(i)],
                dict(
                    mode='immediate',
                    frame=dict(duration=500, redraw=True),
                    transition=dict(duration=0)
                )
            ],
            label=label
        )
        slider_steps.append(step)

    # Keep slider order (left to right)
    sliders = [dict(
        active=len(slider_steps) - 1,  # Start at the newest data
        currentvalue={"prefix": "Start Date: "},
        pad={"t": 50},
        steps=slider_steps
    )]

    fig.update_layout(
        sliders=sliders,
        title='',
        template='plotly_dark',
        width=1000,
        height=400,
        margin=dict(l=20, r=20, t=80, b=150),
        annotations=[
            dict(
                text="Source: <a href='https://fred.stlouisfed.org/'>FRED</a> | Colormap: Job Losers/Leavers: lower is better (green), higher is worse (red). Reentrants/New Entrants: higher is better (green), lower is worse (red).",
                x=0.5,
                y=-0.15,
                xref='paper',
                yref='paper',
                showarrow=False,
                font=dict(size=8, color="white"),
                xanchor='center'
            )
        ]
    )

    return fig


In [10]:
def generate_employment_detailed_table_2(observation_start='2021-01-01'):
    # Initialize FRED API
    fred = Fred(api_key=api_key)  # Replace 'your_api_key' with your actual API key

    # New series for Total Non-Farm Job Openings, Layoffs, Quits, and Hires
    series_ids = {
        'Job Openings: Total Non Farm': 'JTSJOL',
        'Layoff and Discharges: Total Non Farm': 'JTSLDL',
        'Quits: Total Non Farm': 'JTSQUR',
        'Hires: Total Non Farm': 'JTSHIL'
    }

    # Retrieve and process the data
    data = {}
    data_ma = {}  # To store 3mo MA data
    for name, series_id in series_ids.items():
        while True:
            try:
                series_data = fred.get_series(series_id, observation_start=observation_start).resample('M').mean()
                break  # Exit the loop if successful
            except ValueError as e:
                if "Too Many Requests" in str(e):
                    print("Rate limit exceeded. Waiting before retrying...")
                    time.sleep(60)  # Wait for a minute before retrying
                else:
                    raise e
        
        # Convert data to MoM growth rate
        series_data = series_data.pct_change() * 100
        # Calculate 3-month moving average for each series
        series_data_ma = series_data.rolling(window=3).mean()

        data[name] = series_data
        data_ma[name] = series_data_ma

    # Create DataFrames from the retrieved data
    df = pd.DataFrame(data)
    df_ma = pd.DataFrame(data_ma)

    # Drop rows with all NaN values to clean up the DataFrame
    df.dropna(how='all', inplace=True)
    df_ma.dropna(how='all', inplace=True)

    # Transpose the table for the required layout (dates on top, data horizontally)
    df_table = df.T

    # Fill NA values in the table to avoid errors
    df_table.fillna(0, inplace=True)

    # Define window size and step for the slider
    window_size = 30  # months to display
    step_size = 4     # months to step each window
    total_months = len(df_table.columns)
    total_windows = math.ceil((total_months - window_size) / step_size) + 1

    # Prepare the data windows for the slider
    windows = []
    for window_num in range(total_windows):
        start_idx = window_num * step_size
        end_idx = start_idx + window_size
        if end_idx > total_months:
            end_idx = total_months
            start_idx = end_idx - window_size  # Adjust start_idx to have full window
            if start_idx < 0:
                start_idx = 0
        window_dates = df_table.columns[start_idx:end_idx]
        window_data = df_table.iloc[:, start_idx:end_idx]
        windows.append((window_dates, window_data))

    # Generate colors for the heatmap for each row
    def get_row_heatmap_colors(row):
        vmin, vmax = row.min(), row.max()
        return [get_heatmap_color(value, vmin, vmax) for value in row]

    def get_heatmap_color(value, vmin, vmax):
        norm_value = (value - vmin) / (vmax - vmin)
        if value > 0:
            return f'rgba(0, {255 * norm_value}, 0, 0.6)'  # Green for positive values
        else:
            return f'rgba({255 * (1 - norm_value)}, 0, 0, 0.6)'  # Red for negative values

    # Prepare the table and heatmap colors for each window
    def prepare_frame(window_dates, window_data):
        formatted_window_dates = [date.strftime("%y-%m") for date in window_dates]
        table_values = [list(window_data.index)]  # First row: Categories
        for col in window_dates:
            table_values.append(window_data[col].round(1))

        # Generate colors for each row (category)
        colors = [get_row_heatmap_colors(window_data.loc[category]) for category in window_data.index]

        # Flatten the colors for the table
        flat_colors = list(zip(*colors))

        # Prepare the colors for Plotly Table
        color_fill_table = [['black'] * len(window_dates)]  # Header colors
        color_fill_table += flat_colors  # Cell colors

        # Adjust column widths with "Category" taking up 1/5 of the space
        columnwidth = [4] + [0.5] * len(window_dates)
       
        
        table = go.Table(
            columnwidth=columnwidth,
            header=dict(
                values=["Category"] + list(formatted_window_dates),
                fill_color='black',
                font=dict(color='white', size=8),
                align='center',
                height=20  # Set height to 20 as requested
            ),
            cells=dict(
                values=[table_values[0]] + table_values[1:],
                fill=dict(color=color_fill_table),
                font=dict(color='white', size=8),
                align=['left'] + ['center'] * len(formatted_window_dates),
                height=20  # Set height of cells to 20
            )
        )
        return table

    # Create the initial plot
    initial_dates, initial_data = windows[-1]  # Start with the last (newest) window
    initial_table = prepare_frame(initial_dates, initial_data)

    # Create subplots for both graphs and the table
    fig = make_subplots(
        rows=3, cols=1, 
        specs=[[{"type": "scatter"}], [{"type": "scatter"}], [{"type": "table"}]], 
        vertical_spacing=0.1
    )

    # Add the table to the third row
    fig.add_trace(initial_table, row=3, col=1)

    # Add the line plot for the monthly changes (first graph)
    for item in df.columns:
        fig.add_trace(go.Scatter(
            x=df.index, 
            y=df[item], 
            mode='lines',
            name=f'{item} (MoM Change)',
            line=dict(width=2)
        ), row=1, col=1)

    # Add the line plot for the 3-month moving averages (second graph)
    for item in df_ma.columns:
        fig.add_trace(go.Scatter(
            x=df_ma.index, 
            y=df_ma[item], 
            mode='lines',
            name=f'{item} (3mo MA)',
            line=dict(width=2, dash='dash')
        ), row=2, col=1)

    # Create frames for the table to animate with the slider
    frames = []
    for i, (window_dates, window_data) in enumerate(windows):
        frame_table = prepare_frame(window_dates, window_data)
        frame = go.Frame(data=[frame_table], name=str(i))
        frames.append(frame)

    fig.frames = frames

    # Define the slider steps
    slider_steps = []
    for i, (window_dates, _) in enumerate(windows):
        label = window_dates[0].strftime("%y-%m")
        step = dict(
            method='animate',
            args=[
                [str(i)],
                dict(
                    mode='immediate',
                    frame=dict(duration=500, redraw=True),
                    transition=dict(duration=0)
                )
            ],
            label=label
        )
        slider_steps.append(step)

    # Add the slider to the figure (affecting only the table)
    sliders = [dict(
        active=len(slider_steps) - 1,  # Start at the newest data
        currentvalue={"prefix": "Start Date: "},
        pad={"t": 50},
        steps=slider_steps
    )]

    # Update the layout with the slider, centered legend, annotations, and sizing for graphs and table
    fig.update_layout(
        sliders=sliders,
        template='plotly_dark',
        width=1000,
        height=800,  # Increased height to accommodate two graphs and the table
        margin=dict(l=20, r=20, t=80, b=150),
        annotations=[
            dict(
                text="Source: Federal Reserve Economic Data (FRED) <a href='https://fred.stlouisfed.org/'>FRED</a>",
                x=0.5,
                y=-0.15,
                xref='paper',
                yref='paper',
                showarrow=False,
                font=dict(size=8, color="white"),
                xanchor='center'
            )
        ],
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="center",
            x=0.5
        )  # Centered the legend below the title
    )

    return fig


In [11]:
def generate_employment_detailed_table_3():
    series_ids = {
        'Total private': 'CES0500000003',
        'Goods-producing': 'CES0600000003',
        'Mining and logging': 'CES1021000003',
        'Manufacturing': 'CES3000000003',
        'Durable goods': 'CES3100000003',
        'Nondurable goods': 'CES3200000003',
        'Private service-providing': 'CES0800000003',
        'Trade, transportation, and utilities': 'CES4000000003',
        'Information': 'CES5000000003',
        'Financial activities': 'CES5500000003',
        'Professional and business services': 'CES6000000003',
        'Education and health services': 'CES6500000003',
        'Leisure and hospitality': 'CES7000000003',
        'Other services': 'CES8000000003'
    }

    # Fetch all data from FRED
    data_frames = {key: fetch_fred_data(series_id, api_key) for key, series_id in series_ids.items()}

    # Process data to match the required format
    for key, df in data_frames.items():
        if df.empty:
            print(f"No data found for {key}")
            continue
        df['date'] = pd.to_datetime(df['date'])
        df.set_index('date', inplace=True)
        df.rename(columns={'value': key}, inplace=True)
        df[key] = pd.to_numeric(df[key], errors='coerce')
        data_frames[key] = df[key]

    # Merge data into a single DataFrame
    merged_data = pd.concat([df for df in data_frames.values() if not df.empty], axis=1)
    merged_data.sort_index(inplace=True)

    # Calculate the Month-over-Month percentage change
    mom_data = merged_data.pct_change(periods=1) * 100

    # Filter data for a recent time range (e.g., starting from 2021)
    filtered_data = mom_data[mom_data.index >= '2021-01-01']

    # Fixing the date format for the x-axis to YY-MM
    filtered_data.index = filtered_data.index.strftime('%y-%m')

    # Transpose the data for easier plotting
    df_table = filtered_data.T

    # Define window size and step for the slider
    window_size = 20  # Show the last 20 months of data
    step_size = 4     # Step by 4 months
    total_months = len(df_table.columns)
    total_windows = math.ceil((total_months - window_size) / step_size) + 1

    # Create sliding windows of data
    windows = []
    for i in range(total_windows):
        start_idx = i * step_size
        end_idx = start_idx + window_size
        if end_idx > total_months:
            end_idx = total_months
            start_idx = end_idx - window_size
            if start_idx < 0:
                start_idx = 0
        window_dates = df_table.columns[start_idx:end_idx]
        window_data = df_table.iloc[:, start_idx:end_idx]
        windows.append((window_dates, window_data))

    # Use the original colormap logic from your initial code
    def get_row_heatmap_colors(row):
        vmin, vmax = row.min(), row.max()
        return [get_heatmap_color(value, vmin, vmax) for value in row]

    def get_heatmap_color(value, vmin, vmax):
        norm_value = (value - vmin) / (vmax - vmin) if vmax != vmin else 0
        if value > 0:
            return f'rgba(0, {255 * norm_value}, 0, 0.6)'  # Green for positive values
        else:
            return f'rgba({255 * (1 - norm_value)}, 0, 0, 0.6)'  # Red for negative values

    # Prepare the table for each window
    def prepare_frame(window_dates, window_data):
        # Format the dates (headers)
        date_columns = [date for date in window_dates]

        # Apply color mapping to rows
        colors = [get_row_heatmap_colors(row) for row in window_data.values]
        flat_colors = list(map(list, zip(*colors)))

        # Prepare colors for the table
        color_fill = [['black'] * len(date_columns)] * 1  # Header colors
        color_fill += flat_colors  # Cell colors

        # Format categories for the table
        formatted_categories = list(window_data.index)

        # Create the table
        return go.Table(
            columnwidth=[200] + [80 for _ in range(len(date_columns))],
            header=dict(
                values=["Category"] + date_columns,
                fill_color='black',
                font=dict(color='white', size=8),
                align='center'
            ),
            cells=dict(
                values=[formatted_categories] + [window_data[col].round(1) for col in window_data.columns],
                fill=dict(color=color_fill),
                font=dict(color='white', size=8),
                align='left',
                height=30
            )
        )

    # Initialize the figure with the latest window (newest data)
    initial_dates, initial_data = windows[-1]
    initial_table = prepare_frame(initial_dates, initial_data)

    # Create figure
    fig = go.Figure(data=[initial_table])

    # Add animation frames for the slider
    frames = []
    for i, (window_dates, window_data) in enumerate(windows):
        frames.append(go.Frame(data=[prepare_frame(window_dates, window_data)], name=str(i)))

    fig.frames = frames

    # Create slider steps
    slider_steps = []
    for i, (window_dates, _) in enumerate(windows):
        label = window_dates[0]
        slider_steps.append(dict(
            method='animate',
            args=[[str(i)], dict(mode='immediate', frame=dict(duration=500, redraw=True), transition=dict(duration=0))],
            label=label
        ))

    # Create and configure the slider
    sliders = [dict(
        active=len(slider_steps) - 1,
        currentvalue={"prefix": "Start Date: "},
        pad={"t": 50},
        steps=slider_steps
    )]

    # Update layout with the slider and other configuration
    fig.update_layout(
        sliders=sliders,
        width=1000,
        height=650,
        template='plotly_dark',
        margin=dict(l=20, r=20, t=50, b=50),
        annotations=[
            dict(
                text="Source: Federal Reserve Economic Data (FRED) | Note: MoM Percentage Change",
                x=0.5,
                y=-0.01,
                xref='paper',
                yref='paper',
                showarrow=False,
                font=dict(size=8, color="white"),
            )
        ]
    )

    return fig

In [12]:
def generate_employment_detailed_graph_6():
    def process_event_data(event_name, is_percentage=False):
        # Filter event-related data for the specific event_name
        event_data = economic_calendar_data[economic_calendar_data['event'].str.contains(f"{event_name}", case=False)]
        # Convert 'date' to datetime
        event_data['date'] = pd.to_datetime(event_data['date'], format='%d/%m/%Y')

        # Filter for data from 01/01/2020 onward
        event_data = event_data[event_data['date'] >= '2021-01-01']

        # Ensure 'actual' and 'forecast' are strings
        event_data['actual'] = event_data['actual'].astype(str)
        event_data['forecast'] = event_data['forecast'].astype(str)

        # Handle missing or non-numeric values
        if is_percentage:
            event_data['actual'] = event_data['actual'].str.replace('%', '')
            event_data['forecast'] = event_data['forecast'].str.replace('%', '')
        else:
            # Remove commas and any other non-numeric characters
            event_data['actual'] = event_data['actual'].str.replace(',', '').str.extract('([-]?\d+\.?\d*)', expand=False)
            event_data['forecast'] = event_data['forecast'].str.replace(',', '').str.extract('([-]?\d+\.?\d*)', expand=False)

        # Convert 'actual' and 'forecast' to float, handling non-numeric values
        event_data['actual'] = pd.to_numeric(event_data['actual'], errors='coerce')
        event_data['forecast'] = pd.to_numeric(event_data['forecast'], errors='coerce')

        # Drop rows with NaN values in 'actual' or 'forecast'
        event_data.dropna(subset=['actual', 'forecast'], inplace=True)

        # Keep only the most recent event data for each month
        event_data['year_month'] = event_data['date'].dt.to_period('M')
        event_data = event_data.groupby('year_month').apply(lambda x: x.loc[x['date'].idxmax()]).reset_index(drop=True)

        # Calculate the surprise (actual - forecast) / forecast * 100
        event_data['surprise'] = np.where(
            event_data['forecast'] != 0,
            (event_data['actual'] - event_data['forecast']) / event_data['forecast'] * 100,
            np.nan
        )

        # Select the last 15 months
        last_15_months = event_data.tail(30)

        # Create a DataFrame for the table
        df_table = last_15_months[['date', 'actual', 'forecast', 'surprise']].copy()
        df_table.columns = ['Date', 'Actual', 'Forecast', 'Surprise']
        df_table['Date'] = df_table['Date'].dt.strftime('%y-%m')

        df_table.fillna(0, inplace=True)
        df_table_transposed = df_table.set_index('Date').transpose()

        return event_data, df_table_transposed

    def get_row_heatmap_colors(row, reverse=False):
        vmin, vmax = row.min(), row.max()
        return [get_heatmap_color(value, vmin, vmax, reverse) for value in row]

    def get_heatmap_color(value, vmin, vmax, reverse=False):
        if np.isnan(value):
            return 'rgba(100, 100, 100, 0.6)'  # Grey for NaN values
        norm_value = (value - vmin) / (vmax - vmin) if vmax != vmin else 0
        if reverse:  # For cases where higher is worse (e.g., Unemployment Rate)
            if value >= 0:
                return f'rgba({int(255 * norm_value)}, 0, 0, 0.6)'  # Red as the value gets higher (worse)
            else:
                return f'rgba(0, {int(255 * abs(norm_value))}, 0, 0.6)'  # Green as the value gets lower (better)
        else:
            if value >= 0:
                return f'rgba(0, {int(255 * norm_value)}, 0, 0.6)'  # Green as the value gets higher (better)
            else:
                return f'rgba({int(255 * abs(norm_value))}, 0, 0, 0.6)'  # Red as the value gets lower (worse)

    def create_color_fill_per_component(df, reverse=False):
        color_fill = []
        for row in df.values:
            row_colors = get_row_heatmap_colors(row, reverse)
            color_fill.append(row_colors)
        color_fill_transposed = list(zip(*color_fill))
        return color_fill_transposed

    # Process NFP, ADP, and Unemployment data
    adp_data, adp_table = process_event_data("ADP Nonfarm Employment Change")
    nfp_data, nfp_table = process_event_data("Nonfarm Payrolls")
    unem_data, unem_table = process_event_data("Unemployment Rate", is_percentage=True)

    # Create subplots: 8 rows (1 for combined NFP/ADP, 2 for NFP and its table, 2 for ADP and its table, 2 for Unemployment and its table)
    fig = make_subplots(
        rows=8, cols=1,
        specs=[[{'type': 'xy'}], [{'type': 'xy'}], [{'type': 'table'}], [{'type': 'xy'}], [{'type': 'table'}], [{'type': 'xy'}], [{'type': 'table'}], [{'type': 'table'}]],
        subplot_titles=("", "NFP Graph", "NFP Table", "ADP Graph", "ADP Table", "Unemployment Rate Graph", "Unemployment Rate Table"),
        vertical_spacing=0.05  # Adjust vertical spacing between graphs and tables
    )

    # Combined NFP and ADP graph (maximize size for readability)
    fig.add_trace(go.Scatter(x=nfp_data['date'], y=nfp_data['actual'], mode='lines', name='NFP Actual', line=dict(color='cyan')), row=1, col=1)
    fig.add_trace(go.Scatter(x=adp_data['date'], y=adp_data['actual'], mode='lines', name='ADP Actual', line=dict(color='gray')), row=1, col=1)

    # NFP Graph with Forecast (dotted line) (maximize size for readability)
    fig.add_trace(go.Scatter(x=nfp_data['date'], y=nfp_data['actual'], mode='lines', name='NFP Actual', line=dict(color='cyan')), row=2, col=1)
    fig.add_trace(go.Scatter(x=nfp_data['date'], y=nfp_data['forecast'], mode='lines', name='NFP Forecast', line=dict(color='cyan', dash='dot')), row=2, col=1)

    # NFP Table (size fixed to 1000x1000)
    if not nfp_table.empty:
        color_fill_nfp = create_color_fill_per_component(nfp_table)
        fig.add_trace(go.Table(
            header=dict(values=["Item"] + list(nfp_table.columns), fill_color='black', font=dict(color='white', size=8), align='center'),
            cells=dict(values=[nfp_table.index] + [nfp_table[col].round(2) for col in nfp_table.columns],
                       fill=dict(color=[['black'] * len(nfp_table.columns)] + color_fill_nfp), font=dict(color='white', size=8),
                       align='center', height=20)),
            row=3, col=1
        )

    # ADP Graph with Forecast (dotted line) (maximize size for readability)
    fig.add_trace(go.Scatter(x=adp_data['date'], y=adp_data['actual'], mode='lines', name='ADP Actual', line=dict(color='gray')), row=4, col=1)
    fig.add_trace(go.Scatter(x=adp_data['date'], y=adp_data['forecast'], mode='lines', name='ADP Forecast', line=dict(color='gray', dash='dot')), row=4, col=1)

    # ADP Table (size fixed to 1000x1000)
    if not adp_table.empty:
        color_fill_adp = create_color_fill_per_component(adp_table)
        fig.add_trace(go.Table(
            header=dict(values=["Item"] + list(adp_table.columns), fill_color='black', font=dict(color='white', size=8), align='center'),
            cells=dict(values=[adp_table.index] + [adp_table[col].round(2) for col in adp_table.columns],
                       fill=dict(color=[['black'] * len(adp_table.columns)] + color_fill_adp), font=dict(color='white', size=8),
                       align='center', height=20)),
            row=5, col=1
        )

    # Unemployment Graph with Forecast (dotted line) (maximize size for readability)
    fig.add_trace(go.Scatter(x=unem_data['date'], y=unem_data['actual'], mode='lines', name='Unemployment Rate', line=dict(color='lightblue')), row=6, col=1)
    fig.add_trace(go.Scatter(x=unem_data['date'], y=unem_data['forecast'], mode='lines', name='Unemployment Forecast', line=dict(color='lightblue', dash='dot')), row=6, col=1)

    # Unemployment Table (size fixed to 1000x1000)
    if not unem_table.empty:
        color_fill_unem = create_color_fill_per_component(unem_table, reverse=True)  # Reverse for Unemployment
        fig.add_trace(go.Table(
            header=dict(values=["Item"] + list(unem_table.columns), fill_color='black', font=dict(color='white', size=8), align='center'),
            cells=dict(values=[unem_table.index] + [unem_table[col].round(2) for col in unem_table.columns],
                       fill=dict(color=[['black'] * len(unem_table.columns)] + color_fill_unem), font=dict(color='white', size=8),
                       align='center', height=20)),
            row=7, col=1
        )

    # Update layout for better aesthetics
    fig.update_layout(
        title="NFP, ADP, and Unemployment Data",
        width=1000,  # Set width to 1000 pixels
        height=1200,  # Increased height to maximize graph size
        template='plotly_dark',
        margin=dict(l=20, r=20, t=20, b=10),  # Adjusted margins
        font=dict(size=8),  # Set all font sizes to 8
        showlegend=True,
        legend=dict(orientation="h", yanchor="bottom", y=1, xanchor="center", x=0.5),
        title_x=0.5  # Center the title
    )

    return fig

In [13]:
def generate_employment_detailed_graph_7():
    # Initialize FRED API
    fred = Fred(api_key=api_key)
    
    # Fetch all releases of the Nonfarm Payroll Employment (NFP) series
    df = fred.get_series_all_releases('PAYEMS')
    
    # Convert the release date to datetime
    df['realtime_start'] = pd.to_datetime(df['realtime_start'])
    
    # Sort the dataframe by 'date' and 'realtime_start'
    df_sorted = df.sort_values(by=['date', 'realtime_start'])
    
    # Group by 'date' and create a rank for each 'realtime_start' within that date group
    df_sorted['Revision_Label'] = df_sorted.groupby('date')['realtime_start'].rank(method='first').astype(int)
    
    # Pivot the table to have 'date' as index and label each revision as F1, F2, F3, etc.
    df_pivot = df_sorted.pivot(index='date', columns='Revision_Label', values='value')
    
    # Rename the columns as F1, F2, F3, etc.
    df_pivot.columns = [f'F{int(i)}' for i in df_pivot.columns]
    
    # Filter data starting from 2000
    df_pivot = df_pivot[df_pivot.index >= '2000-01-01']
    
    # Calculate the revision difference between F1 and F3 (if F3 exists)
    if 'F3' in df_pivot.columns:
        df_pivot['Revision'] = df_pivot['F3'] - df_pivot['F1']
    else:
        df_pivot['Revision'] = df_pivot['F2'] - df_pivot['F1']
    
    # Calculate the 6-month moving average of the revisions
    df_pivot['6m_MA_Revision'] = df_pivot['Revision'].rolling(window=6).mean()
    
    # Plotting the revision using Plotly
    fig = go.Figure()
    
    # Adding bar plot for actual revisions (F3 - F1)
    fig.add_trace(go.Bar(
        x=df_pivot.index,
        y=df_pivot['Revision'],
        name='Revisions (F3 - F1)',
        marker_color='royalblue'
    ))
    
    # Adding line plot for 6-month moving average
    fig.add_trace(go.Scatter(
        x=df_pivot.index,
        y=df_pivot['6m_MA_Revision'],
        mode='lines',
        name='6-Month Moving Average',
        line=dict(color='tomato', width=2, dash='dash')
    ))
    
    # Update layout for the plot
    fig.update_layout(
        title='NFP Revisions Between Initial and Final Estimates (2000-Present)',
        title_x=0.5,
        xaxis_title='Date',
        yaxis_title='Revision (Thousands of Jobs)',
        showlegend=True,
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=-0.15,  # Position the legend under the title
            xanchor='center',
            x=0.5
        ),
        template='plotly_dark',
        hovermode='x unified',
        yaxis=dict(tickformat=',', showgrid=True),
        xaxis=dict(showgrid=True)
    )
    
    # Return the figure object instead of showing it
    return fig


In [14]:
import pandas as pd
import plotly.graph_objects as go
from fredapi import Fred

def generate_employment_detailed_graph_8():
    # Initialize Fred API with your key
    fred = Fred(api_key=api_key)

    # Fetching data for employment level + job openings (labor demand) and labor force (labor supply)
    labor_demand = fred.get_series('JTSJOL', observation_start='2001-01-01')  # Job Openings
    employment_level = fred.get_series('PAYEMS', observation_start='2001-01-01')  # Nonfarm Employment

    # Labor force (correct measure for labor supply)
    labor_force = fred.get_series('CLF16OV', observation_start='2001-01-01')  # Civilian Labor Force

    # Combining data into a DataFrame for easier manipulation
    df = pd.DataFrame({
        'Labor Demand': employment_level + labor_demand,  # Adding job openings to employment level
        'Labor Supply': labor_force  # Using the labor force as supply
    })

    # Plotting with Plotly
    fig = go.Figure()

    # Labor Demand Line
    fig.add_trace(go.Scatter(
        x=df.index, y=df['Labor Demand'],
        mode='lines',
        name='Labor Demand',
        line=dict(color='blue')
    ))

    # Labor Supply Line
    fig.add_trace(go.Scatter(
        x=df.index, y=df['Labor Supply'],
        mode='lines',
        name='Labor Supply',
        line=dict(color='red')
    ))

    # Layout styling with custom fonts
    fig.update_layout(
        title=dict(text="Labor Demand vs. Labor Supply", font=dict(size=10)),
        xaxis_title=dict(text="Date", font=dict(size=8)),
        yaxis_title=dict(text="Level (Thousands)", font=dict(size=8)),
        template="plotly_dark",
        legend=dict(title=dict(text="Legend", font=dict(size=8)), font=dict(size=8), orientation="h", yanchor="bottom", y=1.02, xanchor="center", x=0.5),
        margin=dict(l=0, r=0, t=50, b=100),  # Add extra bottom margin for the note
    )

    # Add source and note as annotations
    fig.add_annotation(
        text="Source: FRED",
        xref="paper", yref="paper",
        x=0, y=-0.25,  # Position the source below the plot
        showarrow=False,
        font=dict(size=8),
        xanchor='left'
    )

    fig.add_annotation(
        text=("Note: Labor demand = Employment Level + Job Openings. "
              "Labor supply = Civilian Labor Force."),
        xref="paper", yref="paper",
        x=0, y=-0.35,  # Position the note below the source
        showarrow=False,
        font=dict(size=8),
        xanchor='left'
    )

    # Return the Plotly figure
    return fig


In [15]:
def generate_employment_detailed_graph_9():
     
    fred = Fred(api_key='7227512392e5e5d2a2679a261d2bb3a9')
    
    # Fetching data from FRED (starting from 2000)
    job_openings = fred.get_series('JTSJOL', observation_start='2000-01-01')  # Replace with actual FRED ticker for job openings
    corp_profits = fred.get_series('A446RC1Q027SBEA', observation_start='2000-01-01')  # Replace with actual FRED ticker for corporate profits (quarterly)

    # Calculating % YoY change for job openings (monthly data)
    job_openings_yoy = job_openings.pct_change(12) * 100

    # Calculating % YoY change for corporate profits (quarterly data, so use 4-period change)
    corp_profits_yoy = corp_profits.pct_change(4) * 100

    # Shifting corporate profits by 2 months to match timing of job openings
    corp_profits_yoy_shifted = corp_profits_yoy.shift(2)

    # Create a dark theme plotly figure
    fig = go.Figure()

    # Add US Job Openings %YoY (left y-axis)
    fig.add_trace(go.Scatter(
        x=job_openings_yoy.index, 
        y=job_openings_yoy,
        mode='lines',
        name='US Job Openings (%YoY)',
        line=dict(color='white', width=2),
        yaxis='y1'
    ))

    # Add Nonfinancial Corporate Profits %YoY (right y-axis, advanced by 2 months)
    fig.add_trace(go.Scatter(
        x=corp_profits_yoy_shifted.index, 
        y=corp_profits_yoy_shifted,
        mode='lines',
        name='Nonfinancial Corp Profits %YoY (Advanced 2 months)',
        line=dict(color='cyan', width=2),
        yaxis='y2'
    ))

    # Update layout for dual axes and dark theme with increased height
    fig.update_layout(
        title='',
        title_x=0.5,
        template='plotly_dark',
        xaxis=dict(title='Date', range=['2000-01-01', job_openings_yoy.index[-1]]),
        yaxis=dict(
            title='US Job Openings (%YoY)',
            range=[-50, 100],  # LHS y-axis range
            showgrid=True,
            zeroline=True,
            linecolor='white',
            tickcolor='white'
        ),
        yaxis2=dict(
            title='Nonfinancial Corp Profits %YoY (Advanced 2 months)',
            range=[-40, 70],  # RHS y-axis range
            overlaying='y',
            side='right',
            showgrid=False,
            zeroline=False,
            linecolor='cyan',
            tickcolor='cyan'
        ),
        height=700,  # Set the height of the plot here
        legend=dict(
            orientation="h",  # Make the legend horizontal
            x=0.5,  # Center the legend horizontally
            y=1.05,  # Position the legend just below the title
            xanchor='center',  # Center alignment for x position
            yanchor='bottom',  # Align the legend to the bottom of the y position
            font=dict(size=10),
            bgcolor='rgba(0,0,0,0)'
        ),
        annotations=[
            dict(
                text='Source:',
                x=0.5,
                y=-0.015,
                xref='paper',
                yref='paper',
                showarrow=False,
                font=dict(size=10, color="gray"),
                xanchor='center'
            )
        ]
    )

    # Return the figure
    return fig


In [16]:
def generate_employment_detailed_graph_11(observation_start='2020-01-01'):
    # Initialize FRED API
    fred = Fred(api_key=api_key)

    # Fetch data from FRED for IHLIDXUS (Indeed Job Postings Index for the U.S.)
    series_data = fred.get_series('IHLIDXUS', observation_start=observation_start).resample('W').mean()

    # Calculate week-over-week difference and handle potential NaN values
    series_ww_diff = series_data.diff().fillna(0)

    # Create subplots: one for the line plot, one for the table
    fig = make_subplots(
        rows=2, cols=1, 
        row_heights=[0.7, 0.3],  # Adjust the heights to give more space to the chart
        shared_xaxes=True, 
        vertical_spacing=0.1,
        specs=[[{"type": "scatter"}], [{"type": "table"}]]
    )

    # Add the line chart in the first row
    fig.add_trace(
        go.Scatter(
            x=series_data.index, 
            y=series_ww_diff, 
            mode='lines',
            name='WoW Change in Job Postings (IHLIDXUS)',
            line=dict(color='cyan')
        ),
        row=1, col=1
    )

    # Prepare the table data (showing the last 30 WoW changes)
    df_table = series_ww_diff.tail(20).reset_index()
    df_table.columns = ['Date', 'WoW Change']

    # Ensure all data in 'WoW Change' is numeric
    df_table['WoW Change'] = pd.to_numeric(df_table['WoW Change'], errors='coerce').fillna(0)

    # Calculate vmin and vmax explicitly for color differentiation
    vmin, vmax = df_table['WoW Change'].min(), df_table['WoW Change'].max()

    # Prepare colormap for the table
    def get_row_heatmap_colors(row):
        return [get_heatmap_color(value, vmin, vmax) for value in row]

    def get_heatmap_color(value, vmin, vmax):
        norm_value = (value - vmin) / (vmax - vmin) if vmax != vmin else 0
        if value > 0:
            return f'rgba(0, {255 * norm_value}, 0, 0.6)'  # Green for positive values
        else:
            return f'rgba({255 * (1 - norm_value)}, 0, 0, 0.6)'  # Red for negative values

    # Generate colors for each row in the table
    heatmap_colors = get_row_heatmap_colors(df_table['WoW Change'])

    # Transpose the table for dates as headers
    df_table = df_table.T
    df_table.columns = df_table.iloc[0]
    df_table = df_table.drop(df_table.index[0])

    # Add the table with the same formatting, dates as columns and colormap, formatted as YY-MM-DD
    fig.add_trace(
        go.Table(
            header=dict(values=["Category"] + df_table.columns.strftime("%y-%m-%d").tolist(),
                        fill_color='black',
                        font=dict(color='white', size=10),
                        align='center'),
            cells=dict(
                values=[['WoW Change']] + [pd.to_numeric(df_table[col], errors='coerce').round(1).tolist() for col in df_table.columns],
                fill=dict(color=[['black'] * len(df_table.columns)] + [heatmap_colors]),
                font=dict(color='white', size=10),
                align='center')
        ),
        row=2, col=1
    )

    # Update layout to include the subplots and improve aesthetics
    fig.update_layout(
        height=700,  # Adjusted total height for plot and table
        title="Week-over-Week Change in Indeed Job Postings (IHLIDXUS)",
        xaxis_title="Date",
        yaxis_title="WoW Change",
        template='plotly_dark',
        margin=dict(l=20, r=20, t=50, b=20)
    )

    return fig



In [17]:
def generate_employment_detailed_graph_12():
    # Initialize FRED API with your API key
    fred = Fred(api_key=api_key)

    # Define the FRED tickers for each indicator
    indicators = {
        'Total Nonfarm Payrolls Growth Rate': 'PAYEMS',
        'Household Employment Level Growth Rate': 'LNU02000000',
        'Aggregate Weekly Hours Growth Rate': 'AWHAE',
        'Jobless Claims NSA (12M Avg)': 'ICNSA',
        'Insured Unemployment Rate': 'IURNSA',
        'Unemployment Rate': 'UNRATE'
    }

    # Define time periods (the last one will be dynamically set to the most recent date)
    periods = ['18M Ago', '12M Ago', '6M Ago', 'Current']

    # Colors for each period (consistent across all subplots)
    colors = ['#9b59b6', '#f39c12', '#27ae60', '#3498db']  # Purple, Gold, Green, Blue

    # Create an empty dictionary to store FRED data
    data = {}

    # Collect data from FRED for each indicator and dynamically retrieve the most recent date
    most_recent_date = None
    for label, ticker in indicators.items():
        series_data = fred.get_series(ticker)
        if most_recent_date is None:  # Assign the most recent date based on the first series
            most_recent_date = series_data.index[-1].strftime('%Y-%m')  # Format as 'YYYY-MM'
        data[label] = {
            '18M Ago': series_data[-18],
            '12M Ago': series_data[-12],
            '6M Ago': series_data[-6],
            'Current': series_data[-1]  # Use 'Current' for the most recent data point
        }

    # Calculate growth rates for the top 3 indicators (Total Nonfarm, Household Employment, Weekly Hours)
    def calculate_growth_rate(past, current):
        return ((current - past) / past) * 100

    growth_data = {}
    for key in ['Total Nonfarm Payrolls Growth Rate', 'Household Employment Level Growth Rate', 'Aggregate Weekly Hours Growth Rate']:
        growth_data[key] = {
            '18M Ago': calculate_growth_rate(data[key]['18M Ago'], data[key]['12M Ago']),
            '12M Ago': calculate_growth_rate(data[key]['12M Ago'], data[key]['6M Ago']),
            '6M Ago': calculate_growth_rate(data[key]['6M Ago'], data[key]['Current']),
            'Current': calculate_growth_rate(data[key]['6M Ago'], data[key]['Current'])  # most recent rate is over the last 6 months
        }

    # Create a 2x3 grid of subplots
    fig = make_subplots(
        rows=2, cols=3, 
        subplot_titles=("Total Nonfarm Payrolls (Growth Rate)", "Household Employment Level (Growth Rate)", 
                        "Aggregate Weekly Hours (Growth Rate)", "Jobless Claims (NSA) (12M Avg)", 
                        "Insured Unemployment Rate", "Unemployment Rate")
    )

    # Add trace for each period and each indicator
    for i, period in enumerate(periods):
        # Plot Total Nonfarm Payrolls Growth Rate in (row 1, col 1)
        fig.add_trace(go.Bar(
            x=[period], 
            y=[growth_data['Total Nonfarm Payrolls Growth Rate'][period]],
            name=period,  # Use period name for the legend
            marker_color=colors[i],
            text=f"{growth_data['Total Nonfarm Payrolls Growth Rate'][period]:.1f}",
            textposition='outside',
            showlegend=i == 0  # Only show the legend for the first trace of each period
        ), row=1, col=1)

        # Plot Household Employment Level Growth Rate in (row 1, col 2)
        fig.add_trace(go.Bar(
            x=[period], 
            y=[growth_data['Household Employment Level Growth Rate'][period]],
            name=period,
            marker_color=colors[i],
            text=f"{growth_data['Household Employment Level Growth Rate'][period]:.1f}",
            textposition='outside',
            showlegend=False
        ), row=1, col=2)

        # Plot Aggregate Weekly Hours Growth Rate in (row 1, col 3)
        fig.add_trace(go.Bar(
            x=[period], 
            y=[growth_data['Aggregate Weekly Hours Growth Rate'][period]],
            name=period,
            marker_color=colors[i],
            text=f"{growth_data['Aggregate Weekly Hours Growth Rate'][period]:.1f}",
            textposition='outside',
            showlegend=False
        ), row=1, col=3)

        # Plot Jobless Claims NSA (12M Avg) in (row 2, col 1)
        fig.add_trace(go.Bar(
            x=[period], 
            y=[data['Jobless Claims NSA (12M Avg)'][period]],
            name=period,
            marker_color=colors[i],
            text=f"{data['Jobless Claims NSA (12M Avg)'][period]:.0f}",
            textposition='outside',
            showlegend=False
        ), row=2, col=1)

        # Plot Insured Unemployment Rate in (row 2, col 2)
        fig.add_trace(go.Bar(
            x=[period], 
            y=[data['Insured Unemployment Rate'][period]],
            name=period,
            marker_color=colors[i],
            text=f"{data['Insured Unemployment Rate'][period]:.1f}",
            textposition='outside',
            showlegend=False
        ), row=2, col=2)

        # Plot Unemployment Rate in (row 2, col 3)
        fig.add_trace(go.Bar(
            x=[period], 
            y=[data['Unemployment Rate'][period]],
            name=period,
            marker_color=colors[i],
            text=f"{data['Unemployment Rate'][period]:.1f}",
            textposition='outside',
            showlegend=False
        ), row=2, col=3)

    # Update layout to add a legend and remove bar labels
    fig.update_layout(
        title_text="Coincident Employment Indicators",
        template="plotly_dark",
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.2,
            xanchor="center",
            x=0.5,
            title=None  # No title, just the time periods
        ),
        font=dict(size=12),
        margin=dict(t=50, l=50, r=50, b=50),
        height=600,
        width=1000,
        title_x=0.5
    )

    # Update axes titles
    fig.update_xaxes(showticklabels=False)  # Remove x-axis titles
    fig.update_yaxes(title_text="Growth Rate (%)", row=1, col=1, title_font=dict(size=8))
    fig.update_yaxes(title_text="Growth Rate (%)", row=1, col=2, title_font=dict(size=8))
    fig.update_yaxes(title_text="Growth Rate (%)", row=1, col=3, title_font=dict(size=8))
    fig.update_yaxes(title_text="Claims (NSA)", row=2, col=1, title_font=dict(size=8))
    fig.update_yaxes(title_text="Rate (%)", row=2, col=2, title_font=dict(size=8))
    fig.update_yaxes(title_text="Rate (%)", row=2, col=3, title_font=dict(size=8))

    # Set subplot title font size
    for i in fig['layout']['annotations']:
        i['font'] = dict(size=10)  # Set font size to 10 for subplot titles

    return fig



In [18]:
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go

# Initialize Dash app
app = Dash(__name__)

# Add custom CSS to set the entire page background to black and container width to 1000px
app.index_string = '''
<!DOCTYPE html>
<html>
    <head>
        {%metas%}
        <title>{%title%}</title>
        {%favicon%}
        {%css%}
        <style>
            body { 
                background-color: black !important;
                color: white;
                margin: 0;
                padding: 0;
                display: flex;
                justify-content: center;
                align-items: flex-start;
                min-height: 100vh;
            }
            .container {
                width: 1000px; /* Set the width to 1000px */
                padding: 0;
                box-sizing: border-box;
                display: flex;
                justify-content: center;
                align-items: center;
                flex-direction: column;
            }
            .graph-container {
                display: block; /* Ensures the graph takes up full width */
                width: 100%;
                text-align: center; /* Centers the graph itself */
                padding: 0;
                box-sizing: border-box;
                margin: 0 auto; /* Ensures the container is centered */
            }
            .graph-container > div {
                display: inline-block; /* Ensures smaller elements are centered */
                width: 100%;
                padding: 0; /* Remove any internal padding */
                box-sizing: border-box;
                margin: 0 auto; /* Ensures the graphs are centered */
            }
        </style>
    </head>
    <body>
        <div class="container">
            {%app_entry%}
        </div>
        <footer>
            {%config%}
            {%scripts%}
            {%renderer%}
        </footer>
    </body>
</html>
'''

app.layout = html.Div(className='container', children=[
    html.H1("Employment Data Dashboard", style={'textAlign': 'center'}),
    
    dcc.Dropdown(
        id='data-dropdown',
        options=[
            {'label': 'Unemployment Rate', 'value': 'UNEMPLOYMENT'},
        ],
        value='UNEMPLOYMENT',
        style={'backgroundColor': '#000000', 'color': 'white', 'width': '25%', 'margin': '0 auto', 'textAlign': 'center'}
    ),
    
    html.Div(id='employment-graphs', children=[
        html.H2("Unemployment Rate (Graph 1)", style={'textAlign': 'center'}),
        html.Div(className='graph-container', children=[
            dcc.Graph(id='employment-detailed-graph-1', style={'width': '100%', 'max-width': '1000px'})
        ]),
        
        html.H2("BLS Employment Situation (Change in Thousands SA Graph 2)", style={'textAlign': 'center'}),
        html.Div(className='graph-container', children=[
            dcc.Graph(id='employment-detailed-graph-2', style={'width': '100%', 'max-width': '1000px'})
        ]),
        
        html.H2("ADP Employment Situation (Change in Thousands SA Graph 3)", style={'textAlign': 'center'}),
        html.Div(className='graph-container', children=[
            dcc.Graph(id='employment-detailed-graph-3', style={'width': '100%', 'max-width': '1000px'})
        ]),
        
       
         html.H2("Surpirse Indicators (Graph 4)", style={'textAlign': 'center'}),
        html.Div(className='graph-container', children=[
            dcc.Graph(id='employment-detailed-graph-4', style={'width': '100%', 'max-width': '1000px'})
        ]),
        
        
         html.H2("Total Nonfarm Payrolls and U3 Unemployment Rate (Graph 5)", style={'textAlign': 'center'}),
        html.Div(className='graph-container', children=[
            dcc.Graph(id='employment-detailed-graph-5', style={'width': '100%', 'max-width': '1000px'})
        ]),
        

        html.H2("Indeed Job Posting (WoW % Graph 6)", style={'textAlign': 'center'}),
        html.Div(className='graph-container', children=[
            dcc.Graph(id='employment-detailed-graph-6', style={'width': '100%', 'max-width': '1000px'})
        ]),

        
        html.H2("Wage Growth Analysis (Graph 7)", style={'textAlign': 'center'}),
        html.Div(className='graph-container', children=[
            dcc.Graph(id='employment-detailed-graph-7', style={'width': '100%', 'max-width': '1000px'})
        ]),

         html.H2("Unemployed persons by reason of unemployment (Monthly SA In Thousands Table 1)", style={'textAlign': 'center'}),
        html.Div(className='graph-container', children=[
            dcc.Graph(id='employment-detailed-table-1', style={'width': '100%', 'max-width': '1000px'})
        ]),

        
        html.H2("Job Openings and Labor Turnover Survey (Table 2)", style={'textAlign': 'center'}),
        html.Div(className='graph-container', children=[
            dcc.Graph(id='employment-detailed-table-2', style={'width': '100%', 'max-width': '1000px'})
        ]),

        html.H2("Average Hourly Earnings from Employment Situation (MoM % Growth SA Table 3)", style={'textAlign': 'center'}),
        html.Div(className='graph-container', children=[
            dcc.Graph(id='employment-detailed-table-3', style={'width': '100%', 'max-width': '1000px'})
        ]),

        html.H2("Fed Funds Rate and Employment (Graph 8)", style={'textAlign': 'center'}),
        html.Div(className='graph-container', children=[
            dcc.Graph(id='employment-detailed-graph-8', style={'width': '100%', 'max-width': '1000px'})
        ]),

        html.H2("Rising Corporate Profits should support labour demand (Graph 9)", style={'textAlign': 'center'}),
        html.Div(className='graph-container', children=[
            dcc.Graph(id='employment-detailed-graph-9', style={'width': '100%', 'max-width': '1000px'})
        ]),
        
        html.H2("Job Postings vs Job Openings (Graph 10)", style={'textAlign': 'center'}),
        html.Div(className='graph-container', children=[
            dcc.Graph(id='employment-detailed-graph-10', style={'width': '100%', 'max-width': '1000px'})
        ]),


        # Adding Graph 11, 12, and 13
        html.H2("Graph 11 (Title here)", style={'textAlign': 'center'}),
        html.Div(className='graph-container', children=[
            dcc.Graph(id='employment-detailed-graph-11', style={'width': '100%', 'max-width': '1000px'})
        ]),

        html.H2("Graph 12 (Title here)", style={'textAlign': 'center'}),
        html.Div(className='graph-container', children=[
            dcc.Graph(id='employment-detailed-graph-12', style={'width': '100%', 'max-width': '1000px'})
        ]),

        html.H2("Graph 13 (Title here)", style={'textAlign': 'center'}),
        html.Div(className='graph-container', children=[
            dcc.Graph(id='employment-detailed-graph-13', style={'width': '100%', 'max-width': '1000px'})
        ]),
    ], style={'display': 'block'})
])

# Callbacks for Employment Graphs 1-13
@app.callback(
    Output('employment-detailed-graph-1', 'figure'),
    [Input('data-dropdown', 'value')]
)
def update_employment_detailed_graph_1(selected_data):
    return generate_employment_detailed_graph_1() if selected_data == 'UNEMPLOYMENT' else go.Figure()

# Similar callbacks for Graphs 2-13
@app.callback(
    Output('employment-detailed-graph-2', 'figure'),
    [Input('data-dropdown', 'value')]
)
def update_employment_detailed_graph_2(selected_data):
    return generate_employment_detailed_graph_2() if selected_data == 'UNEMPLOYMENT' else go.Figure()


@app.callback(
    Output('employment-detailed-table-1', 'figure'),
    [Input('data-dropdown', 'value')]
)
def update_employment_detailed_table_1(selected_data):
    return generate_employment_detailed_table_1() if selected_data == 'UNEMPLOYMENT' else go.Figure()

# Similar callbacks for Graphs 2-13
@app.callback(
    Output('employment-detailed-table-2', 'figure'),
    [Input('data-dropdown', 'value')]
)
def update_employment_detailed_table_2(selected_data):
    return generate_employment_detailed_table_2() if selected_data == 'UNEMPLOYMENT' else go.Figure()


# Similar callbacks for Graphs 2-13
@app.callback(
    Output('employment-detailed-table-3', 'figure'),
    [Input('data-dropdown', 'value')]
)
def update_employment_detailed_table_3(selected_data):
    return generate_employment_detailed_table_3() if selected_data == 'UNEMPLOYMENT' else go.Figure()


@app.callback(
    Output('employment-detailed-graph-3', 'figure'),
    [Input('data-dropdown', 'value')]
)
def update_employment_detailed_graph_3(selected_data):
    return generate_employment_detailed_graph_3() if selected_data == 'UNEMPLOYMENT' else go.Figure()

@app.callback(
    Output('employment-detailed-graph-4', 'figure'),
    [Input('data-dropdown', 'value')]
)
def update_employment_detailed_graph_4(selected_data):
    return generate_employment_detailed_graph_4() if selected_data == 'UNEMPLOYMENT' else go.Figure()

@app.callback(
    Output('employment-detailed-graph-5', 'figure'),
    [Input('data-dropdown', 'value')]
)
def update_employment_detailed_graph_5(selected_data):
    return generate_employment_detailed_graph_5() if selected_data == 'UNEMPLOYMENT' else go.Figure()

@app.callback(
    Output('employment-detailed-graph-6', 'figure'),
    [Input('data-dropdown', 'value')]
)
def update_employment_detailed_graph_6(selected_data):
    return generate_employment_detailed_graph_6() if selected_data == 'UNEMPLOYMENT' else go.Figure()

@app.callback(
    Output('employment-detailed-graph-7', 'figure'),
    [Input('data-dropdown', 'value')]
)
def update_employment_detailed_graph_7(selected_data):
    return generate_employment_detailed_graph_7() if selected_data == 'UNEMPLOYMENT' else go.Figure()

@app.callback(
    Output('employment-detailed-graph-8', 'figure'),
    [Input('data-dropdown', 'value')]
)
def update_employment_detailed_graph_8(selected_data):
    return generate_employment_detailed_graph_8() if selected_data == 'UNEMPLOYMENT' else go.Figure()

@app.callback(
    Output('employment-detailed-graph-9', 'figure'),
    [Input('data-dropdown', 'value')]
)
def update_employment_detailed_graph_9(selected_data):
    return generate_employment_detailed_graph_9() if selected_data == 'UNEMPLOYMENT' else go.Figure()

@app.callback(
    Output('employment-detailed-graph-10', 'figure'),
    [Input('data-dropdown', 'value')]
)
def update_employment_detailed_graph_10(selected_data):
    return generate_employment_detailed_graph_10() if selected_data == 'UNEMPLOYMENT' else go.Figure()

@app.callback(
    Output('employment-detailed-graph-11', 'figure'),
    [Input('data-dropdown', 'value')]
)
def update_employment_detailed_graph_11(selected_data):
    return generate_employment_detailed_graph_11() if selected_data == 'UNEMPLOYMENT' else go.Figure()

@app.callback(
    Output('employment-detailed-graph-12', 'figure'),
    [Input('data-dropdown', 'value')]
)
def update_employment_detailed_graph_12(selected_data):
    return generate_employment_detailed_graph_12() if selected_data == 'UNEMPLOYMENT' else go.Figure()

@app.callback(
    Output('employment-detailed-graph-13', 'figure'),
    [Input('data-dropdown', 'value')]
)
def update_employment_detailed_graph_13(selected_data):
    return generate_employment_detailed_graph_13() if selected_data == 'UNEMPLOYMENT' else go.Figure()

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True, port=8067)


/var/folders/lg/dbmt0tnx1h74vny5m2398l980000gr/T/ipykernel_72992/2689285243.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/lg/dbmt0tnx1h74vny5m2398l980000gr/T/ipykernel_72992/2689285243.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/lg/dbmt0tnx1h74vny5m2398l980000gr/T/ipykernel_72992/2689285243.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
Cell In[18], line 175, in update_employment_detailed_graph_1(selected_data='UNEMPLOYMENT')
    170 @app.callback(
    171     Output('employment-detailed-graph-1', 'figure'),
    172     [Input('data-dropdown', 'value')]
    173 )
    174 def update_employment_detailed_graph_1(selected_data):
--> 175     return generate_employment_detailed_graph_1() if selected_data == 'UNEMPLOYMENT' else go.Figure()
        selected_data == 'UNEMPLOYMENT' = True
        selected_data = 'UNEMPLOYMENT'
        go = <module 'plotly.graph_objects' from '/opt/anaconda3/lib/python3.11/site-packages/plotly/graph_objects/__init__.py'>

Cell In[4], line 10, in generate_employment_detailed_graph_1()
      8 def generate_employment_detailed_graph_1():
      9     # Load data from CSV
---> 10     data = load_employment_data()
     12     # Filter data from January

/opt/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



Error fetching data for series CES1021000003: {'error_code': 400, 'error_message': 'Bad Request.  The series does not exist.'}


/var/folders/lg/dbmt0tnx1h74vny5m2398l980000gr/T/ipykernel_72992/2689285243.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/lg/dbmt0tnx1h74vny5m2398l980000gr/T/ipykernel_72992/2689285243.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/lg/dbmt0tnx1h74vny5m2398l980000gr/T/ipykernel_72992/2689285243.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

...

---------------------------------------------------------------------------
HTTPError                                 Traceback (most recent call last)
File /opt/anaconda3/lib/python3.11/site-packages/fredapi/fred.py:84, in Fred.__fetch_data(
    self=<fredapi.fred.Fred object>,
    url='https://api.stlouisfed.org/fred/series/observati...22-01-01&api_key=7227512392e5e5d2a2679a261d2bb3a9'
)
     83 try:
---> 84     response = urlopen(url)
        url = 'https://api.stlouisfed.org/fred/series/observations?series_id=PAYEMS&start_date=2022-01-01&api_key=7227512392e5e5d2a2679a261d2bb3a9'
     85     root = ET.fromstring(response.read())

File /opt/anaconda3/lib/python3.11/urllib/request.py:216, in urlopen(
    url='https://api.stlouisfed.org/fred/series/observati...22-01-01&api_key=7227512392e5e5d2a2679a261d2bb3a9',
    data=None,
    timeout=<object object>,
    cafile=None,
    capath=None,
    cadefault=False,
    context=None
)
    215     opener = _opener
--> 216 return opener.open(url

/var/folders/lg/dbmt0tnx1h74vny5m2398l980000gr/T/ipykernel_72992/2689285243.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/lg/dbmt0tnx1h74vny5m2398l980000gr/T/ipykernel_72992/2689285243.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/lg/dbmt0tnx1h74vny5m2398l980000gr/T/ipykernel_72992/2689285243.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
Cell In[18], line 175, in update_employment_detailed_graph_1(selected_data='UNEMPLOYMENT')
    170 @app.callback(
    171     Output('employment-detailed-graph-1', 'figure'),
    172     [Input('data-dropdown', 'value')]
    173 )
    174 def update_employment_detailed_graph_1(selected_data):
--> 175     return generate_employment_detailed_graph_1() if selected_data == 'UNEMPLOYMENT' else go.Figure()
        selected_data == 'UNEMPLOYMENT' = True
        selected_data = 'UNEMPLOYMENT'
        go = <module 'plotly.graph_objects' from '/opt/anaconda3/lib/python3.11/site-packages/plotly/graph_objects/__init__.py'>

Cell In[4], line 10, in generate_employment_detailed_graph_1()
      8 def generate_employment_detailed_graph_1():
      9     # Load data from CSV
---> 10     data = load_employment_data()
     12     # Filter data from January

/var/folders/lg/dbmt0tnx1h74vny5m2398l980000gr/T/ipykernel_72992/2689285243.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/lg/dbmt0tnx1h74vny5m2398l980000gr/T/ipykernel_72992/2689285243.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/lg/dbmt0tnx1h74vny5m2398l980000gr/T/ipykernel_72992/2689285243.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

---------------------------------------------------------------------------
HTTPError                                 Traceback (most recent call last)
File /opt/anaconda3/lib/python3.11/site-packages/fredapi/fred.py:84, in Fred.__fetch_data(
    self=<fredapi.fred.Fred object>,
    url='https://api.stlouisfed.org/fred/series/observati...99-12-31&api_key=7227512392e5e5d2a2679a261d2bb3a9'
)
     83 try:
---> 84     response = urlopen(url)
        url = 'https://api.stlouisfed.org/fred/series/observations?series_id=PAYEMS&realtime_start=1776-07-04&realtime_end=9999-12-31&api_key=7227512392e5e5d2a2679a261d2bb3a9'
     85     root = ET.fromstring(response.read())

File /opt/anaconda3/lib/python3.11/urllib/request.py:216, in urlopen(
    url='https://api.stlouisfed.org/fred/series/observati...99-12-31&api_key=7227512392e5e5d2a2679a261d2bb3a9',
    data=None,
    timeout=<object object>,
    cafile=None,
    capath=None,
    cadefault=False,
    context=None
)
    215     opener = _opener
--